## Bayes Lab 3

The list data contains 100 numbers drawn from a normal distribution with unknown model parameters $\lambda$ and $\mu$.

$$\frac{dP}{d\lambda_L}(x) = \sqrt{\frac{\lambda}{2 \pi}} e^{\frac{-\lambda}{2}(x - \mu)^2}$$

**Exercise:**

1. Use the quadratic loss to estimate the model parameters $\lambda$ and $\mu$. Plot and justify each assumption that you are making. 
2. Determine appropriate confidence intervalls. 
3. Are $\mu$ and $\lambda$ correlated? $COV[\lambda,\mu] = \frac{E[(\lambda - E[\lambda])(\mu - E[\mu])]}{\sqrt{Var[\lambda]Var[\mu]}} <> 0$
4. Do a little bit of research and determine the estimates for $\lambda$ and $\mu$ under the absolut loss $L_1(\theta,\delta) = |

In [ ]:
data = [  1.68620459e+01,   1.22860390e+01,   1.42572671e+01,
         1.14752971e+01,   1.10697986e+01,   1.16259813e+01,
         1.28204955e+01,   1.32841907e+01,   8.91244195e+00,
         1.49721838e+01,   1.03022139e+01,   1.47209438e+01,
         1.19125043e+01,   1.08553119e+01,   4.64457775e+00,
         1.19747994e+01,   1.32684660e+01,   1.14822467e+01,
         8.93703138e+00,   1.21580146e+01,   2.67317145e+00,
         6.96649263e+00,   7.98445237e+00,   3.59013492e+00,
         8.64985882e+00,   5.59262784e+00,   9.30891167e+00,
         1.32405497e+01,   1.45620358e+01,   9.56935522e+00,
         9.52189436e+00,   1.39756263e+01,   1.82570194e+01,
         9.81023967e+00,   1.50505281e+01,   1.71825330e+01,
         1.50031113e+01,   1.32739666e+01,   5.82516430e+00,
         1.09788719e+01,   1.38452615e+01,   6.53772710e+00,
         9.36198335e+00,   1.25787750e+01,   1.70038050e+01,
         9.97551713e+00,   9.97286445e+00,   1.73503796e+01,
         1.09519968e+01,   8.95443490e+00,   1.08568883e+01,
         7.67177454e+00,   9.16088790e+00,   1.09589633e+01,
         1.04563748e+01,   1.70943984e+01,   8.82388913e+00,
         1.40835232e+01,   1.27480329e+01,   8.48409699e+00,
         7.95601637e+00,   1.09277835e+01,   1.66272979e+01,
         7.19547058e+00,   1.53242086e+01,   6.29073988e+00,
         1.25335659e+01,   9.64268750e+00,   8.78983657e+00,
         1.22289117e+01,   9.76756105e+00,   1.31259362e+01,
         1.31720782e+01,   8.39355478e+00,   8.21438973e+00,
         1.83671307e+01,   1.12473511e+01,   9.51763021e+00,
         1.36521061e+01,   1.32737361e+01,   1.24579796e+01,
         1.26594200e+01,   1.19790087e+01,   1.20928438e+01,
         7.61966506e+00,   1.37482389e+01,   1.63331047e+01,
         6.86156896e+00,   1.14146867e+01,   1.30367025e+01,
         1.28356329e+01,  -5.24528821e-03,   5.92999169e+00,
         1.40478409e+01,   9.03228272e+00,   9.50347334e+00,
         1.15757948e+01,   1.31938005e+01,   1.05835812e+01,
         7.96437213e+00]

In [ ]:
import math
import numpy as np
from matplotlib import pyplot as plt
from scipy.special import gamma
import emcee
%matplotlib inline

a,b,c,d = 1.0,1.2,1.4,1.5
pd = lambda l,m: np.prod([math.sqrt(l/(2*np.pi))*np.exp((-l/2)*((x-m)**2)) for x in data])
ng = lambda l,m: (((d**c)*math.sqrt(b))/(gamma(c)*math.sqrt(2*np.pi)))\
                    * (m**(c-0.5))*(np.exp(-d*m))*(np.exp(-1*(b*m*((l-a)**2)))/2)

post = np.vectorize(lambda l,m: pd(l,m)*ng(l,m))

datasum = sum(data)
datalen = len(data)
dataavg = datasum/datalen

In [ ]:
def lnprob(l):
    if l < 0:
        return -np.infty
    else:
        return (datasum + 5-1)*np.log(l) - (datalen+1)*l #log of post goes here

nwalkers = 200 
ndim = 2
p0 = np.random.rand(nwalkers*ndim).reshape((nwalkers,ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)
pos, prob, state = sampler.run_mcmc(p0, 1000)
sampler.reset()
pos, prob, state = sampler.run_mcmc(pos, 100000)
samples = sampler.flatchain 

In [ ]:
import pandas as pd
data = pd.DataFrame(samples)
data.plot(kind ='density')

In [ ]:
def MonteCarlo(f, samples):
    N = len(samples)
    return 1/float(N)*sum([f(e) for e in samples])

Exp = MonteCarlo(lambda x: x, samples)
Exp2 = MonteCarlo(lambda x: x**2, samples)
var = np.sqrt(Exp2 - Exp**2)
print('Expectation Value:{0}, 3sigma:{1}'.format(Exp, 3*var))